In [1]:
import pandas as pd
# import numpy as np

In [2]:
database_type = "/full"

The `patient` table includes general information about the patient admissions (for example, demographics, admission and discharge details). 
See: http://eicu-crd.mit.edu/eicutables/patient/

In [3]:
# loads patient table
patient = pd.read_csv('../eICU' + database_type + '/patient.csv', nrows=999999) 
patient = patient[['patientunitstayid', 'admissionweight']].set_index('patientunitstayid')
patient

,admissionweight
patientunitstayid,
141168,84.3
141178,54.4
141179,NaN
141194,73.9
141196,NaN
...,...
3353235,90.0
3353237,78.4
3353251,102.0


The `vitalperiodic` table comprises data that is consistently interfaced from bedside vital signs monitors into eCareManager. 
Data are generally interfaced as 1 minute averages, and archived into the `vitalperiodic` table as 5 minute median values. 
For more detail, see: http://eicu-crd.mit.edu/eicutables/vitalPeriodic/

In [4]:
vitalperiodic = pd.read_csv('../eICU' + database_type + '/vitalPeriodic.csv', nrows=999999)
columns = ['patientunitstayid', 'observationoffset', 'temperature', 'heartrate', 'respiration', 'systemicsystolic']
vitalperiodic = vitalperiodic[columns]
vitalperiodic = vitalperiodic.sort_values(by='observationoffset')
vitalperiodic

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic
653065,148349,-1445,NaN,60.0,20.0,NaN
653071,148349,-1440,NaN,60.0,21.0,NaN
790455,150049,-1435,NaN,96.0,19.0,NaN
479374,146418,-1435,NaN,71.0,23.0,NaN
653145,148349,-1435,NaN,60.0,21.0,NaN
...,...,...,...,...,...,...
273524,144297,71778,NaN,72.0,33.0,NaN
273140,144297,71783,NaN,72.0,27.0,NaN
273956,144297,71788,NaN,72.0,34.0,NaN
275114,144297,71793,NaN,72.0,33.0,NaN


The `lab` table provides Laboratory tests that have have been mapped to a standard set of measurements. 
Unmapped measurements are recorded in the customLab table.
See: http://eicu-crd.mit.edu/eicutables/lab/

In [67]:
lab = pd.read_csv('../eICU' + database_type + '/lab.csv', nrows=999999)
columns = ['patientunitstayid', 'labresultoffset','labtypeid','labname', 'labresult', 'labmeasurenamesystem', 'labmeasurenameinterface']
lab = lab[columns]
lab = lab.sort_values(by='labresultoffset')
lab

,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labmeasurenamesystem,labmeasurenameinterface
337586,154910,-73277,4,bedside glucose,77.00,mg/dL,mg/dL
337274,154910,-73231,3,MCH,24.90,pg,pg
337258,154910,-73231,3,-monos,2.00,%,%
337290,154910,-73231,3,platelets x 1000,76.00,K/mcL,K/mcL
337298,154910,-73231,3,WBC x 1000,2.90,K/mcL,K/mcL
...,...,...,...,...,...,...,...
861141,174525,144155,3,-basos,0.00,%,%
860866,174525,144155,1,BUN,12.00,mg/dL,mg/dL
861160,174525,144155,3,RBC,3.87,M/mcL,mil/mcL
860818,174525,144155,1,glucose,105.00,mg/dL,mg/dL


In [68]:
#converting labname 'urinary creatinine to 'creatinine'
lab.loc[lab['labname'].str.contains("urinary creatinine"), 'labname'] = "creatinine"
lab.loc[(lab['labname'].str.contains("creatinine"))]

lab = lab.drop(['labtypeid', 'labmeasurenamesystem', 'labmeasurenameinterface'], axis = 1)

labc = lab.copy()
labw = lab.copy()
labl = lab.copy()

labc = labc.loc[(labc['labname'].str.contains("creatinine"))]
labw = labw.loc[(labw['labname'].str.contains("WBC x 1000"))]
labl = labl.loc[(labl['labname'].str.contains("lactate"))]

In [69]:
# Pull out dataframe for labresults 'WBC x 1000', 'lactate', 'creatinine'
lab_name = ['WBC x 1000', 'lactate', 'creatinine']
lab = lab[lab.labname.isin(lab_name)]
labc['creatinine'] = labc.apply(lambda x: x['labresult'] if x['labname'] == 'creatinine' else -1, axis=1)
labw['wbcx1000'] = labw.apply(lambda x: x['labresult'] if x['labname'] == 'WBC x 1000' else -1, axis=1)
labl['lactate'] = labl.apply(lambda x: x['labresult'] if x['labname'] == 'lactate' else -1, axis=1)

labc = labc.drop(['labname', 'labresult'], axis = 1)
labw = labw.drop(['labname', 'labresult'], axis = 1)
labl = labl.drop(['labname', 'labresult'], axis = 1)
labl

,patientunitstayid,labresultoffset,lactate
612542,165083,-24915,1.6
612411,165083,-24237,1.5
243022,151167,-20878,1.3
364105,155885,-18625,2.0
573344,163473,-14459,1.0
...,...,...,...
683712,167896,50708,0.9
339288,154911,51080,6.6
338118,154911,51880,10.0
683922,167896,73168,1.9


In [71]:
flab = labc.merge(labw, how = 'left', on = ['patientunitstayid', 'labresultoffset'])
flab = flab.merge(labl, how = 'left', on = ['patientunitstayid', 'labresultoffset'])
flab['lactate'].unique()

,patientunitstayid,labresultoffset,creatinine,wbcx1000,lactate
0,154910,-72872,5.40,5.4,NaN
1,154910,-71858,6.07,8.3,NaN
2,154910,-70836,3.61,6.7,NaN
3,154910,-69363,2.86,5.5,NaN
4,154910,-67941,3.67,5.0,NaN
...,...,...,...,...,...
30639,157644,130238,3.30,5.5,NaN
30640,157644,131628,4.34,6.9,NaN
30641,157644,133088,3.10,6.1,NaN
30642,174525,137425,0.64,11.1,NaN


The `intakeOutput` table provides Intake and output recorded for patients entered from the nursing flowsheet.
See: http://eicu-crd.mit.edu/eicutables/intakeOutput/

In [7]:
intakeOutput = pd.read_csv('../eICU' + database_type + '/intakeOutput.csv', nrows=999999)
columns = ['patientunitstayid', 'intakeoutputoffset', 'intaketotal', 'outputtotal', 'celllabel']
intakeOutput = intakeOutput[columns]
intakeOutput.sort_values(by='intakeoutputoffset')
intakeOutputUrine = intakeOutput.loc[(intakeOutput['celllabel'].str.contains("Urine"))]
intakeOutputUrine = pd.DataFrame.merge(patient, intakeOutputUrine, on='patientunitstayid')
intakeOutputUrine

,patientunitstayid,admissionweight,intakeoutputoffset,intaketotal,outputtotal,celllabel
0,141179,NaN,1420,0.0,700.0,Urine
1,141179,NaN,933,0.0,500.0,Urine
2,141179,NaN,1703,0.0,1000.0,Urine
3,141179,NaN,678,0.0,700.0,Urine
4,141194,73.90,12201,290.4,100.0,Urine
...,...,...,...,...,...,...
207342,425009,65.80,11046,0.0,300.0,Urine
207343,425010,65.80,1358,0.0,150.0,Urine
207344,425072,93.90,28,0.0,350.0,Urine
207345,425072,93.90,1108,0.0,0.0,Urine


In [8]:
intakeOutputUrine['admissionweight'].fillna(intakeOutputUrine['admissionweight'].mean(), inplace=True)
intakeOutputUrine['urineoutputbyweight'] = intakeOutputUrine.apply(lambda x: x['outputtotal'] / x['admissionweight'], axis=1)
intakeOutputUrine = intakeOutputUrine[['patientunitstayid','intakeoutputoffset', 'urineoutputbyweight']]
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,urineoutputbyweight
0,141179,1420,7.851544
1,141179,933,5.608246
2,141179,1703,11.216491
3,141179,678,7.851544
4,141194,12201,1.353180
...,...,...,...
207342,425009,11046,4.559271
207343,425010,1358,2.279635
207344,425072,28,3.727370
207345,425072,1108,0.000000


In [9]:
# Merging Final Training Table
# merged_df = vitalperiodic.merge(lab, how='inner', left_on=["A", "B"], right_on=["A2","B2"])
